In [ ]:
from pathlib import Path
import pandas as pd, numpy as np

In [ ]:
base_folder = Path("/home/julienb/Downloads/RTXI1--box1--1709_2309--birdGold4--fs20000/")
result_folder = Path("/media/filer2/T4b/Birds/Songs")
files = list(base_folder.glob("**/*.txt"))
files

In [ ]:
df = pd.DataFrame()
def get_info(name, pattern):
    import re
    matches= list(re.finditer(pattern, name))
    if len(matches) ==0:
        return None
    elif len(matches)==1:
        return matches[0]["info"]
    else:
        raise Exception("multiple matches")
df["txt"] = files
df["box"] =  get_info(base_folder.name, '(--|^)box(?P<info>\w+)(--|$)')
df["bird"] =  get_info(base_folder.name, '(--|^)bird(?P<info>\w+)(--|$)')
df["fs"] =  float(get_info(base_folder.name, '(--|^)fs(?P<info>(\d|\.)+)(--|$)'))
df["date_epoch"] = df["txt"].apply(lambda path: int(path.stem[5:]))
df

In [ ]:
import datetime
df["date"] = df["date_epoch"].apply(lambda num: datetime.datetime.fromtimestamp(num))
df = df.sort_values("date")
df

In [ ]:
df["dest_path"] = str(result_folder) + "/" + df["bird"] + "/" + df["date"].dt.strftime("%Y-%m-%d") + "/" +  df["bird"] + "--box" + df["box"] + "--" + df["date"].dt.strftime("%Y-%m-%d_%H-%M-%S")+ ".wav"
if df["dest_path"].duplicated().any():
    print(df[df["dest_path"].duplicated(keep=False)].to_string())
    raise Exception(f'duplication...')
print(df.to_string())

In [ ]:
from tqdm.auto import tqdm
for _, row in tqdm(df.iterrows(), total=len(df.index)):
    txt, fs, dest = row["txt"], row["fs"], Path(row["dest_path"])
    if dest.exists(): raise Exception(f"File {dest} already exists....")
    data = np.loadtxt(txt).reshape(-1)
    import scipy.io, scipy.signal
    if fs > 20000:
        filter = scipy.signal.butter(3, [100, 10000], btype="bandpass", output="sos", fs=fs)
    else:
        filter = scipy.signal.butter(3, 100, btype="highpass", output="sos", fs=fs)
    filtered_data = scipy.signal.sosfiltfilt(filter, data)
    if fs - int(fs) > 0.0000001:
        aligned_filtered_data = np.interp(np.arange(filtered_data.size)/np.round(fs), np.arange(filtered_data.size)/fs, filtered_data)
        new_fs = int(np.round(fs))
    else:
        aligned_filtered_data = filtered_data
        new_fs = int(fs)
    dest.parent.mkdir(exist_ok=True, parents=True)
    scipy.io.wavfile.write(dest, new_fs, aligned_filtered_data)